# Engineering Formulas
Mechanical / piping engineering calculations.

In [1]:
import numpy as np
import pandas as pd

# Basics

---
## Velocity / Diameter / Flowrate

Simply formula manipulation to make it easy to switch between velocity / diameter and flowrate.

**Formula**
$$ Q = AV  $$

**Parameters**
- A : float
    - Area of pipe, [m²]
- V : float
    - Velocity, [m/s]

**Returns**
- Q : float
    - Flowrate, [m³/s]

**Python Formula**

In [4]:
def find_dia(Q_m3h, V_ms):
    area = Q_m3h / 60 / 60 / V_ms
    dia_mm = np.sqrt((4*area)/(np.pi)) * 1000
    return dia_mm

def find_flow(dia_mm, V_ms):
    area = (np.pi*(dia_mm/1000)**2)/4
    Q_m3h = V_ms * area * 60 * 60
    return Q_m3h

def find_vel(dia_mm, Q_m3h):
    area = (np.pi*(dia_mm/1000)**2)/4
    V_ms = Q_m3h / 60 / 60 / area
    return V_ms

**Example**

In [11]:
find_vel(dia_mm = 254, Q_m3h = 400)

2.19280582376665

# Fluid Mechanics

## Bernoulli's principle

Bernoulli's principle states that an increase in the speed of a fluid occurs simultaneously with a decrease in static pressure or a decrease in the fluid's potential energy.

**Formula**
$$ P_1 + \frac{1}{2} \rho v_1^2 +\rho g h = 
  P_2 + \frac{1}{2} \rho v_2^2 +\rho g h $$
  
**Parameters**
- P : float
   - Pressure Energy, [Pa]
- $\rho$ : float
   - Density of the fluid, [kg/m³]
- v : float
   - Velocity of fluid, [m/s]
- h : float
  - Physical Height, [m]
- g : float
    - Gravitational acceleration = 9.81, [m/s²]

In [4]:
def pumphead (delta_h, fric_f, L_m, d_mm , Ktotal, V, P1, P2, rho):
        return ((P2-P1)*1000/(rho*g)) + (delta_h) + (( fric_f * (L/(d/1000)) + Ktotal ) * ((V**2)/(2*g)))

**References**

[1] Yunus Cengel, Fluid Mechanics. eq. (8–55). 4th edition. 2017.

---
## Friction loss through pipe

**Formula**
$$ h_L = f\frac{L}{D}\frac{V^2}{2g}  $$
**Parameters**
- f : float
    - Darcy Friction Factor, [-]
- L : float
    - Length of pipe, [m]
- D : float
    - Inner diameter of pipe, [m]
- V : float
    - Velocity of fluid, [m/s]
- g : float
    - Gravitational acceleration = 9.81, [m/s²]

**Returns**
- h_l : float
    - Head loss, [m]

**Python Formula**

In [5]:
def friction_loss(f, L, D, V, g = 9.81):
    h_l = f * (L/D) * ((V**2) / (2*g))
    return h_l

**Example**

In [6]:
friction_loss(0.019, 1000, 0.254, 2.3)

20.168637177233574

**References**

[1] Yunus Cengel, Fluid Mechanics. eq. (8-24). 4th edition. 2017.

---
## Flow Discharge Factor - K Factor

**Formula**
$$ K_L = \frac{h_L}{V^2/(2g)} $$

**Parameters**
- h_L : float
    - Head losses through fitting, [m]
- V : float
    - Velocity of fluid, [m/s]
- g : float
    - Gravitational acceleration = 9.81, [m/s²]

**Returns**
- K_L : float
    - Flow Discharge Factor, [-]
    
**Python Formula**

In [7]:
def k_factor(h_L, V, g = 9.81):
  K_L = (h_L*2*g)/(V**2)
  return K_L

**Example**

In [8]:
h = 1790E3 / (1234 * 9.81)

v = find_vel(70.2, 419.5)

k_factor(h, v)


3.2006423647444993

**References**

[1] Yunus Cengel, Fluid Mechanics. eq. (8–55). 4th edition. 2017.

---
## Relationship Between Flow Coefficient and Resistance Coefficient

**Formula**
$$ K = 0.00214\frac{d^4}{C_v^2} $$
**Parameters**
- d : float
    - Diameter, [mm]
- C_v : float
    - Flow coefficient, [gal/min]

**Returns**
- K : float
    - Flow Discharge Factor, [-]

**Python Equation**

In [9]:
def flow_to_resist_coef(d, C_v):
    K = 0.00214 * ((d**4) / (C_v**2))
    return K

**Example**

In [10]:
flow_to_resist_coef(72.8, 140)

3.0667842150399993

**Reference**

[Relationship Between Flow Coefficient and Resistance Coefficient](http://kb.eng-software.com/eskb/ask-an-engineer/theory-equations-and-calculated-results-questions/relationship-between-flow-coefficient-and-resistance-coefficient)

---
## Open Channel Flow

**Summary**
- Generally target 50% fillage
- No single mathematical model to represent slurry flows in open channels
- Max practical limit of slurry concentration is 65 wt%

**Dimensions**

Hydraulic Radius = area of the flow : wetted perimeter
$$R_H = \frac{A}{P}$$
Hydraulic Diameter
$$D_H = \frac{4A}{P}$$

** Hydraulic Diameter **
![hydraulic_radius](images/hyd_rad.png)

### Froude Number
- Is a measure of the stability of flow in an open channel. Ratio of the fluids' inertia to gravity forces.

**Slurry**
- Below 1.5 there is risk of settling and instability / wavy motion
- Above 5 there is risk of [[Slug flow]] which causes instability in the form of roll waves

$$1.5 < F_r < 5$$

**Water**
- Sub-critical flow for water (Fr less than 0.8) is not a problem as there is no risk of settling. Therefore:

$$F_r < 0.8 $$ or $$F_r > 1.2$$

**Equation**

$$F_r =\frac{V}{\sqrt{g*y_m}}$$

**Parameters**
- y_m : float
    - Average depth of the liquid, [m]
- V : float
    - Velocity of fluid, [m/s]
- g : float
    - Gravitational acceleration = 9.81, [m/s²]

**Returns**
- Fr : float
    -  Froude Number, [-]

**Python Equation**


In [8]:
def froude_no(V, y_m, g = 9.81):
    return V / np.sqrt(g * y_m)

**References**

[1]: SLURRY SYSTEMS HANDBOOK. BAHA E. ABULNAGA, P.E. 2002

### Fillage Calculation

In [9]:
def fillage_calc(Q_m3h, H1_highspot, H2_lowspot, Length_pipe, ID_m, f_d):
   
    h_delta = H1_highspot-H2_lowspot
    r = ID_m/2         # radius
    A_pipe = (np.pi * ID_m ** 2)/4
    df = pd.DataFrame(data= {'theta_deg': np.linspace(0.1,180,1800)})
    df['theta'] = np.deg2rad(df['theta_deg'])
    df['R_h'] = (r*(df['theta']-np.sin(df['theta'])*np.cos(df['theta']))
                 )/(2*df['theta'])
    df['A_c'] = r**2*(df['theta']-np.sin(df['theta'])*np.cos(df['theta']))
    df['y'] = r*(1-np.cos(df['theta']))     # Height of the liquid
    df['fillage'] = df['y']/ ID_m *100      # % fillage
    df['V_L'] = (Q_m3h/60/60)/(df['A_c'])   # Velocity of the liquid
    df['h_fric'] = (f_d * Length_pipe * df['V_L'] **2 )/(2*9.81*4*df['R_h'])
        
    # Find closest match of fricion loss to the elevation difference
    result_index = df['h_fric'].sub(h_delta).abs().idxmin()
    # Check Froude Number
    Fr = df['V_L'][result_index] / np.sqrt(9.81 * df['y'][result_index])
    fillage = df['fillage'][result_index]
    
    # Assuming that the velocity of the slurry is equal to the air velocity
    Q_air = df['V_L'][result_index] * A_pipe * 60 * 60 - Q_m3h
    
    results = pd.DataFrame({'fillage %':[fillage],
                            'Froude No.':[Fr],
                            'Velocity':[df['V_L'][result_index]],
                            'Vent Air Flow Req. m3/h':[Q_air]})
    return results

In [10]:
fillage_calc(200, 20, 0, 100, 0.254, 0.05)

,fillage %,Froude No.,Velocity,Vent Air Flow Req. m3/h
0,32.898993,4.225445,3.825737,497.870642
